**Maturity-risk for variational hedging**

This notebook is used to code and test numerical solutions of Forward-Backward Stochastic Differential Equation Systens (FBSDE) obtained through the minimization of composed risk measures that solves optimal hedging under variational preferences approach.

**Algorithm for Determining the Optimal Hedge $h_t^*$**

We consider the optimal hedging problem in incomplete markets formulated on the probability space $(\Omega, \mathcal{F}, \{\mathcal{F}_t\}_{t\in[0,T]}, \mathbb{P})$, where the underlying asset $S_t$ follows a given stochastic dynamics (GBM, Heston, or Jump-Diffusion). Let $H_t = H(S_t,t)$ be the value of the replicated derivative, $L_t$ the profit process associated with the hedging strategy $h_t$, and $\rho_u$ the composite risk-utility function defining the cost functional. The objective is to find $h_t^*$ that minimizes the terminal risk.

---

**Problem Formulation**

The admissible control $h_t$ belongs to a set $\mathcal{A}$ of progressively measurable processes satisfying integrability conditions. The problem is

$$
\inf_{h \in \mathcal{A}} J(h) := \inf_{h \in \mathcal{A}} \rho_u(L_T^h)
$$

where $L_T^h$ is the terminal profit induced by the control $h$.

---

**Forward-Backward System**

The problem is described by the following FBSDE:

*Forward Equation (asset and profit)*

$$
\begin{cases}
dS_t = \mu(t,S_t) S_t \, dt + \sigma(t,S_t) S_t \, dW_t + S_t^- dJ_t, \\
dL_t = b(t,S_t,h_t) dt + \eta(t,S_t,h_t) dW_t + \xi(t,S_t,h_t) dJ_t, \\
S_0 = s_0, \quad L_0 = 0.
\end{cases}
$$

*Backward Equation (adjoint process)*

$$
\begin{cases}
dp_t = -\nabla_L \mathcal{H}(t,S_t,L_t,h_t,p_t,q_t,r_t) \, dt + q_t dW_t + r_t d\tilde{J}_t, \\
p_T = \partial g(L_T),
\end{cases}
$$

where $g$ is the terminal cost defined by the composite risk function $\rho_u$.

---

**Hamiltonian**

The Hamiltonian of the problem is defined as

$$
\mathcal{H}(t,S,L,h,p,q,r) =
p \, b(t,S,h) + q \, \eta(t,S,h) + r \, \xi(t,S,h),
$$

where $p_t$, $q_t$, $r_t$ are adjoint processes.

The maximum condition of the **Stochastic Maximum Principle (SMP)** requires

$$
\frac{\partial \mathcal{H}}{\partial h}(t,S,L,h_t^*,p_t,q_t,r_t) = 0.
$$

---

**Structure of the Optimal Control**

In the cases considered in this work, the explicit solution of the maximum condition has the form

$$
h_t^* = \frac{\partial H}{\partial S}(S_t,t) + \Phi(S_t, p_t, q_t, r_t),
$$

where:
- $\partial H/\partial S$ is the **classical delta hedge**,
- $\Phi$ is the **variational adjustment term** depending on the risk/utility function.

---

**Backward Numerical Algorithm**

Let $t_0, t_1, \dots, t_N=T$ be a partition of $[0,T]$ with step size $\Delta t$. The algorithm is as follows:

*Step 1 – Initialization*

1. Set an initial strategy $h^{(0)}_t$ (e.g., delta hedge).  
2. Simulate $M$ trajectories of the asset $S_t$ and compute the corresponding profit $L_t$.

---

*Step 2 – Picard Iteration*

For each iteration $k=0,1,2,\dots$:

1. **Forward**: simulate the system  
   $$
   S_{n+1}^{(k)} = S_n^{(k)} + \mu S_n^{(k)} \Delta t + \sigma S_n^{(k)} \Delta W_n,
   $$
   $$
   L_{n+1}^{(k)} = L_n^{(k)} + b(t_n, S_n^{(k)}, h_n^{(k)}) \Delta t
     + \eta(t_n, S_n^{(k)}, h_n^{(k)}) \Delta W_n.
   $$

2. **Backward**: solve the BSDE for $p_t^{(k)}$:
   $$
   p_N^{(k)} = \partial g(L_T^{(k)}),
   $$
   $$
   p_n^{(k)} = \mathbb{E} \big[ p_{n+1}^{(k)} + f(t_{n+1}, S_{n+1}^{(k)}, L_{n+1}^{(k)}, p_{n+1}^{(k)}) \Delta t \mid \mathcal{F}_{t_n} \big],
   $$
   where $f = -\nabla_L \mathcal{H}$.  
   The conditional expectation is approximated using Monte Carlo regression.

3. **Computation of $q_n^{(k)}$**:
   $$
   q_n^{(k)} = \frac{1}{\Delta t} \mathbb{E} \big[ (p_{n+1}^{(k)} - p_n^{(k)}) \Delta W_n^\top \mid \mathcal{F}_{t_n} \big].
   $$

4. **Control Update**:
   $$
   h_n^{(k+1)} = \frac{\partial H}{\partial S}(S_n^{(k)}, t_n) + \Phi(S_n^{(k)}, p_n^{(k)}, q_n^{(k)}, r_n^{(k)}).
   $$

---

*Step 3 – Convergence Criterion*

Repeat Step 2 until

$$
\max_n \| h_n^{(k+1)} - h_n^{(k)} \| < \varepsilon.
$$

Upon convergence, define $h_t^* = h_t^{(k+1)}$.

---

**Result**

The algorithm above produces a sequence of admissible controls $\{h^{(k)}\}$ that converges to the optimal control $h^*$ minimizing the risk criterion. The numerical solution combines:

- **Forward simulation** of the asset and profit,  
- **Backward solution** of the adjoint BSDE,  
- **Iterative update** of the control via the Hamiltonian.


**Algorithm 1: Forward Simulation of the Process $S_t$**

**Input:** Initial value $S_0$, parameters $\mu$, $\sigma$, time step $\Delta t$, Brownian increments $\Delta W_n$, number of steps $N$, number of simulated paths $M$  
**Output:** Simulated asset paths $S_t^{(m)}$ for $m = 1, \ldots, M$

---

**begin**

1. for each path $m = 1, \ldots, M$ do  
2.   initialize $S_0^{(m)} \leftarrow S_0$  
3. end  

4. for $n = 0, \ldots, N-1$ do  
5.   for each path $m = 1, \ldots, M$ do  
6.     simulate Brownian increment $\Delta W_n^{(m)} \sim \mathcal{N}(0, \Delta t)$  
7.     update asset price  
8.       $S_{n+1}^{(m)} \leftarrow S_n^{(m)} + \mu S_n^{(m)} \Delta t + \sigma S_n^{(m)} \Delta W_n^{(m)}$  
9.   end  
10. end  

**end**

11. return $S_t^{(m)}$ for $m = 1, \ldots, M$  


**Algorithm 2: Backward Simulation of the Process $H_t$**  

**Input:** Asset paths $S_t$, Brownian increments $\Delta W_n$, time step $\Delta t$, terminal payoff $H_T = (S_T - K)^+$  
**Output:** Paths $H_t$, estimates $\partial_S H_t$, $\partial_{SS} H_t$, $\partial_t H_t$

---

**begin**

1. for each path $m = 1, \ldots, M$ do  
2.   $H_N^{(m)} \leftarrow (S_N^{(m)} - K)^+$  
3. end  

4. for $n = N-1, \ldots, 0$ do  
5.   for each path $m = 1, \ldots, M$ do  
6.     estimate $\alpha_n^{(m)}$, $\beta_n^{(m)}$ by solving  
7.       $\min_{\alpha, \beta} \sum [ H_{n+1}^{(m)} - H_n^{(m)} - \alpha \Delta t - \beta \Delta W_n^{(m)} ]^2$  
8.     update  
9.       $H_n^{(m)} \leftarrow H_{n+1}^{(m)} - \alpha_n^{(m)} \Delta t - \beta_n^{(m)} \Delta W_n^{(m)}$  
10.    compute  
11.      $\partial_S H_n^{(m)} \leftarrow \beta_n^{(m)} / (\sigma S_n^{(m)})$  
12.      $\partial_{SS} H_n^{(m)} \leftarrow [2(\alpha_n^{(m)} - \mu S_n^{(m)} \partial_S H_n^{(m)})] / [\sigma^2 (S_n^{(m)})^2]$  
13.      $\partial_t H_n^{(m)} \leftarrow [H_{n+1}^{(m)} - H_n^{(m)}]/\Delta t - \mu S_n^{(m)} \partial_S H_n^{(m)} - 0.5 \sigma^2 (S_n^{(m)})^2 \partial_{SS} H_n^{(m)}$  
14.   end  
15. end  

**end**

16. return $H_t^{(m)}$, $\partial_S H_t^{(m)}$, $\partial_{SS} H_t^{(m)}$, $\partial_t H_t^{(m)}$  



**Algorithm 3: Forward Simulation of the Profit Process $L_t$**

**Input:** Simulated asset paths $S_t^{(m)}$, control $h_n^{(m)}$, time step $\Delta t$, Brownian increments $\Delta W_n^{(m)}$, parameters $\mu$, $\sigma$, derivative estimates $\partial_S H_n^{(m)}$, $\partial_{SS} H_n^{(m)}$, $\partial_t H_n^{(m)}$, number of steps $N$, number of simulated paths $M$  
**Output:** Simulated profit process $L_t^{(m)}$ for $m = 1, \ldots, M$

---

**begin**

1. for each path $m = 1, \ldots, M$ do  
2.   initialize $L_0^{(m)} \leftarrow 0$  
3. end  

4. for $n = 0, \ldots, N-1$ do  
5.   for each path $m = 1, \ldots, M$ do  
6.     compute drift coefficient  
7.       $b_n^{(m)} \leftarrow (h_n^{(m)} - \partial_S H_n^{(m)}) \mu S_n^{(m)} - \partial_t H_n^{(m)} - 0.5 \sigma^2 (S_n^{(m)})^2 \partial_{SS} H_n^{(m)}$  
8.     compute diffusion coefficient  
9.       $\eta_n^{(m)} \leftarrow (h_n^{(m)} - \partial_S H_n^{(m)}) \sigma S_n^{(m)}$  
10.    update profit  
11.       $L_{n+1}^{(m)} \leftarrow L_n^{(m)} + b_n^{(m)} \Delta t + \eta_n^{(m)} \Delta W_n^{(m)}$  
12.   end  
13. end  

**end**

14. return $L_t^{(m)}$ for $m = 1, \ldots, M$  


**Algorithm 4: Backward Solution of the BSDE for $p_t^{(k)}$**

**Input:** Simulated asset paths $S_t^{(k,m)}$, profit paths $L_t^{(k,m)}$, time step $\Delta t$, number of steps $N$, number of simulated paths $M$  
**Output:** Adjoint process $p_t^{(k,m)}$, auxiliary process $q_n^{(k,m)}$

---

**begin**

1. for each path $m = 1, \ldots, M$ do  
2.   set terminal condition $p_N^{(k,m)} \leftarrow \partial g(L_T^{(k,m)})$  
3. end  

4. for $n = N-1, \ldots, 0$ do  
5.   for each path $m = 1, \ldots, M$ do  
6.     compute  
7.       $$p_n^{(k,m)} \leftarrow \mathbb{E}\Big[ p_{n+1}^{(k,m)} + f(t_{n+1}, S_{n+1}^{(k,m)}, L_{n+1}^{(k,m)}, p_{n+1}^{(k,m)}) \Delta t \,\Big|\, \mathcal{F}_{t_n} \Big]$$  
8.     where $f = -\nabla_L \mathcal{H}$.  
9.     approximate the conditional expectation using Monte Carlo regression.  
10.   end  
11. end  

12. for $n = 0, \ldots, N-1$ do  
13.   compute $q_n^{(k)}$ as  
14.     $$q_n^{(k)} \leftarrow \frac{1}{\Delta t} \mathbb{E}\Big[ (p_{n+1}^{(k)} - p_n^{(k)}) \Delta W_n^\top \,\Big|\, \mathcal{F}_{t_n} \Big]$$  
15. end  

**end**

16. return $p_t^{(k,m)}$, $q_n^{(k,m)}$  


**Algorithm 5: Computation of $q_n^{(k)}$ and $r_n^{(k)}$**

**Input:** Adjoint process $p_t^{(k,m)}$, Brownian increments $\Delta W_n^{(m)}$, Poisson jump increments $\Delta N_n^{(m)}$, time step $\Delta t$, number of steps $N$, number of simulated paths $M$, jump intensity $\lambda$  
**Output:** Processes $q_n^{(k,m)}$ and $r_n^{(k,m)}$

---

**begin**

1. for $n = 0, \ldots, N-1$ do  
2.   for each path $m = 1, \ldots, M$ do  
3.     compute $q_n^{(k,m)}$ as  
4.       $$q_n^{(k,m)} \leftarrow \frac{1}{\Delta t} \mathbb{E}\Big[ (p_{n+1}^{(k,m)} - p_n^{(k,m)}) \Delta W_n^{(m)\top} \,\Big|\, \mathcal{F}_{t_n} \Big]$$  
5.     if jumps are present then  
6.       compute $r_n^{(k,m)}$ as  
7.       $$r_n^{(k,m)} \leftarrow \frac{1}{\lambda \Delta t} \mathbb{E}\Big[ (p_{n+1}^{(k,m)} - p_n^{(k,m)}) \Delta N_n^{(m)} \,\Big|\, \mathcal{F}_{t_n} \Big]$$  
8.   end  
9. end  

**end**

10. return $q_n^{(k,m)}$, $r_n^{(k,m)}$  


**Algorithm 6: Control Update and Convergence Criterion**

**Input:** Estimates $\partial_S H_n^{(m)}$, state variables $S_n^{(k,m)}$, adjoint process $p_n^{(k,m)}$, auxiliary processes $q_n^{(k,m)}$, $r_n^{(k,m)}$, tolerance $\varepsilon$  
**Output:** Updated control $h_n^{(k+1)}$, optimal control $h_t^*$

---

**begin**

1. for $n = 0, \ldots, N$ do  
2.   for each path $m = 1, \ldots, M$ do  
3.     update the control  
4.       $$h_n^{(k+1)} = \partial_S H_n^{(m)}(S_n^{(k)}, t_n) + \Phi(S_n^{(k)}, p_n^{(k)}, q_n^{(k)}, r_n^{(k)})$$  
5.   end  
6. end  

---

**Step 3 – Convergence Criterion**

7. Repeat Step 2 until  
8. $$
   \max_n \big\| h_n^{(k+1)} - h_n^{(k)} \big\| < \varepsilon.
   $$

9. Upon convergence, define  
10. $$
    h_t^* = h_t^{(k+1)}.
    $$

**end**

11. return $h_t^*$
